In [1]:
import numpy as np
import pandas as pd

# STEPS

Resampling

Log transformation

Simple Lags

Diff based lags


In [11]:
df=pd.read_excel('dell_data.xlsx',
              skiprows=2,index_col='Date')
df.index=pd.to_datetime(df.index)
df.columns=[i.lower().replace('&','_') for i in df.columns]
df.head(3)#data Price of dell smapled at single day


#Data gatherer, collceted from request based apis
#Index set as datecolumn


,dell,s_p500
Date,,
2012-05-23,12.49,1318.86
2012-05-24,12.45,1320.68
2012-05-25,12.46,1317.82


In [70]:
# I will define trsanformers for each function
# Resampling 
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline


class resampler(BaseEstimator,TransformerMixin):
  def __init__(self,freq,cols_tbu=None):
    self.cols_tbu=cols_tbu
    self.freq=freq
  
  def fit(self,dt,y=None):
    return
  
  def transform(self,dt,y=None):
    assert(type(dt) in [type(pd.DataFrame()), type(pd.Series())])
    assert(any(((dt.reset_index()).dtypes.astype(str)).str.contains('datetime')))

    dt=dt.resample(rule=self.freq).last()
    dt.ffill(inplace=True)
    return dt

temp=resampler(freq='5D')
temp.transform(df).head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  app.launch_new_instance()


,dell,s_p500
Date,,
2012-05-23,12.46,1317.82
2012-05-28,12.07,1278.04
2012-06-02,12.22,1315.13


In [88]:
class log_transformer(BaseEstimator,TransformerMixin):
  dt_fn={
      'e':np.log,
      'e1':np.log10
  }

  def __init__(self,base='e',cols_tbu=None):
    self.cols_tbu=cols_tbu
    self.fn=self.dt_fn[base]
  
  def fit(self,dt,y=None):
    return
  
  def transform(self,dt,y=None):
    dt=self.fn(dt)

    return dt

temp=log_transformer(base='e1')
temp.transform(df).head(3)

,dell,s_p500
Date,,
2012-05-23,1.096562,3.120199
2012-05-24,1.095169,3.120798
2012-05-25,1.095518,3.119856


In [112]:
class lag_transformer(BaseEstimator,TransformerMixin):

  def __init__(self,lt_lags,cols_tbu=None):
    self.cols_tbu=cols_tbu
    self.lt_lags=list(set([0]+lt_lags))
  
  def fit(self,dt,y=None):
    return
  
  def transform(self,dt,y=None):
    parent_columns=dt.columns
    dt_lagged=pd.concat([dt.shift(lag).dropna().rename(columns={i: i+'_lagged_'+str(lag) for i in parent_columns
                                                      })
                       for lag in self.lt_lags],axis=1).dropna()
    return dt_lagged

temp=lag_transformer(lt_lags=[1,2,5])
temp.transform(df).head(3)

,dell_lagged_0,s_p500_lagged_0,dell_lagged_1,s_p500_lagged_1,dell_lagged_2,s_p500_lagged_2,dell_lagged_5,s_p500_lagged_5
Date,,,,,,,,
2012-05-31,12.33,1310.33,12.56,1313.32,12.66,1332.42,12.49,1318.86
2012-06-01,12.07,1278.04,12.33,1310.33,12.56,1313.32,12.45,1320.68
2012-06-04,12.00,1278.18,12.07,1278.04,12.33,1310.33,12.46,1317.82


In [114]:
pip=Pipeline([
              ('resample',resampler('3D')),
              ('log_transformer',log_transformer(base='e1')),
              ('lag_trasformer',lag_transformer(lt_lags=[1,2,5]))
])

pip.transform(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  app.launch_new_instance()


,dell_lagged_0,s_p500_lagged_0,dell_lagged_1,s_p500_lagged_1,dell_lagged_2,s_p500_lagged_2,dell_lagged_5,s_p500_lagged_5
Date,,,,,,,,
2012-06-07,1.083503,3.122432,1.087071,3.118969,1.081707,3.106544,1.095518,3.119856
2012-06-10,1.078094,3.121947,1.083503,3.122432,1.087071,3.118969,1.095518,3.119856
2012-06-13,1.089905,3.128024,1.078094,3.121947,1.083503,3.122432,1.090963,3.117381
2012-06-16,1.094122,3.128651,1.089905,3.128024,1.078094,3.121947,1.081707,3.106544
2012-06-19,1.079181,3.122383,1.094122,3.128651,1.089905,3.128024,1.087071,3.118969
2012-06-22,1.086360,3.125488,1.079181,3.122383,1.094122,3.128651,1.083503,3.122432
2012-06-25,1.090963,3.124455,1.086360,3.125488,1.079181,3.122383,1.078094,3.121947
2012-06-28,1.097257,3.134228,1.090963,3.124455,1.086360,3.125488,1.089905,3.128024
2012-07-01,1.103119,3.137993,1.097257,3.134228,1.090963,3.124455,1.094122,3.128651


# ROUGH WORK

In [71]:
type(pd.DataFrame()) in [type(pd.DataFrame()),type(pd.DataFrame())]

True

In [53]:
type(df.index)
(df.reset_index()).dtypes
# (df.index).dtype

# (df.reset_index()).selct_dtypes

(df.reset_index()).select_dtypes(include=['datetime64'])
# 'datetime64' in 
any(((df.reset_index()).dtypes.astype(str)).str.contains('datetime'))

True

In [67]:
df.resample('2D').last().ffill()
pd.DataFrame.resample?

In [89]:
tt=np.log10
tt(1e3)


3.0

In [111]:
lt_lag=[1,2,5]
lt_lag=[0]+lt_lag
pd.concat([df.shift(lag).dropna().rename(columns={i: i+'_lagged_'+str(lag) for i in df.columns
            })
 for lag in lt_lag],axis=1).dropna()

,dell_lagged_0,s_p500_lagged_0,dell_lagged_1,s_p500_lagged_1,dell_lagged_2,s_p500_lagged_2,dell_lagged_5,s_p500_lagged_5
Date,,,,,,,,
2012-05-31,12.33,1310.33,12.56,1313.32,12.66,1332.42,12.49,1318.86
2012-06-01,12.07,1278.04,12.33,1310.33,12.56,1313.32,12.45,1320.68
2012-06-04,12.00,1278.18,12.07,1278.04,12.33,1310.33,12.46,1317.82
2012-06-05,12.16,1285.50,12.00,1278.18,12.07,1278.04,12.66,1332.42
2012-06-06,12.22,1315.13,12.16,1285.50,12.00,1278.18,12.56,1313.32
2012-06-07,12.13,1314.99,12.22,1315.13,12.16,1285.50,12.33,1310.33
2012-06-08,12.12,1325.66,12.13,1314.99,12.22,1315.13,12.07,1278.04
2012-06-11,11.86,1308.93,12.12,1325.66,12.13,1314.99,12.00,1278.18
2012-06-12,11.97,1324.18,11.86,1308.93,12.12,1325.66,12.16,1285.50


# Reference

https://stackoverflow.com/questions/9542738/python-find-in-list
